# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D,CuDNNLSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers
from keras.layers import LSTM

from keras.layers.noise import GaussianNoise

import tensorflow as tf

Using TensorFlow backend.


# Define architectures

Define each one of the different architectures to be tested.

In [2]:
K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.10

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
#     #model.add(Masking(mask_value=0, imput))
#     #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
#     model.add(LSTM(input_shape=input_shape, units=150, dropout=0.2, recurrent_dropout=0.2, return_sequences=False, name='lstm1'))
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
#     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', name='out'))
    #model.add(LSTM(input_shape=input_shape, units=150, dropout=0.1, recurrent_dropout=0.2, return_sequences=False, name='lstm1'))
    model.add(CuDNNLSTM(input_shape=input_shape, units=150, return_sequences=False, name='lstm1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', name='out'))
    
    return model

# def RULmodel_SN_4(input_shape):
#     #Create a sequential model
#     model = Sequential()
# #     model.add(GaussianNoise( 0.02 ))
#     model.add(Dropout(0.2))
#     #Add the layers for the model
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
# #                     name='fc1'))
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
# #                     name='fc2'))
# #     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))

#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc1'))
#     #model.add(Dropout(0.2))
#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc2'))
    
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
# #                     name='fc3'))
    
#     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
#     return model

In [3]:
def RULmodel_SN_1(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(100, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_3(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

In [4]:
def get_compiled_model(model_def, shape, model_type='ann'):
    
    K.clear_session()

    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = model_def(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

In [5]:
#Define the usable models for this notebook

models = {'lstm-20-10':RULmodel_LSTM}
#models = {'shallow-16-16-12': RULmodel_SN_4}

#models = {'shallow-250-100':RULmodel_SN_4, 'shallow-100-50':RULmodel_SN_1, 'shallow-50-20':RULmodel_SN_2,
#          'shallow-20-20':RULmodel_SN_3, 'shallow-20':RULmodel_SN_5, 'shallow-10':RULmodel_SN_6}

# Process Data

In [6]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, 
                                  window_size, window_stride)

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = StandardScaler()



# Load Data

In [7]:
#For LSTM
#tModel.data_handler.data_scaler = scaler
#tModel.data_scaler = None

#For ANN
# tModel.data_handler.data_scaler = None
# tModel.data_scaler = scaler

# tModel.data_handler.sequence_length = 24
# tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
# tModel.data_handler.sequence_stride = 1
# tModel.data_handler.max_rul = 129

# tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.3)
# tModel.print_data()

# Test on dataset 1

In [8]:
iterations = 10
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
num_features = len(selected_features)

windowSize = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
windowStride = 1
constRul = 125

file = open("results/LSTM/1layerdrop0.2.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

tModel = SequenceTunableModelRegression('lstm-20-10', None, lib_type='keras', data_handler=dHandler_cmaps)
tModel.epochs = 100

#For LSTM
tModel.data_handler.data_scaler = scaler
tModel.data_scaler = None

#For ANN
# tModel.data_handler.data_scaler = None
# #tModel.data_scaler = min_max_scaler
# tModel.data_scaler = scaler

for key, model_def in models.items():
  
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
  
    for i in range(1,2):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')
      
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        cv_tempScoresRMSE = np.zeros((iterations,1))
        cv_tempScoresRHS = np.zeros((iterations,1))
        cv_tempTime = np.zeros((iterations,1))
      
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = windowSize
        tModel.data_handler.sequence_stride = windowStride
        tModel.data_handler.max_rul = constRul
        #tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0.2)
        #for lstm
        tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0.2)
        #tModel.print_data()
        
        #input_shape = tModel.data_handler.sequence_length*num_features #For simple ANN
        input_shape = (tModel.data_handler.sequence_length,num_features) #For RNN

        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            #model = get_compiled_model(model_def, input_shape, model_type='ann')
            model = get_compiled_model(model_def, input_shape, model_type='lstm')

            tModel.change_model(key, model, 'keras')
            #tModel.train_model(verbose=0, learningRate_scheduler=lrate)
            tModel.train_model(verbose=0) #lstm
            print("CV set")
            tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
          
            cv_tempScoresRMSE[j] = rmse2
            cv_tempScoresRHS[j] = rhs
            cv_tempTime[j] = time
            
            print("Test set")
            tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
          
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time

        print("Results for cross validtion model " + key)
  
        print(stats.describe(cv_tempScoresRMSE))
        print(stats.describe(cv_tempScoresRHS))
        print(stats.describe(cv_tempTime))
        
        cv_tempScoresRMSE = np.reshape(cv_tempScoresRMSE, (iterations,))
        cv_tempScoresRHS = np.reshape(cv_tempScoresRHS, (iterations,))
        cv_tempTime = np.reshape(cv_tempTime, (iterations,))
        
        csvfile.writerow(cv_tempScoresRMSE)
        csvfile.writerow(cv_tempScoresRHS)
        csvfile.writerow(cv_tempTime)

        print("Results for test model " + key)
  
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
          
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        
        
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
        
file.close()

For model lstm-20-10
Computing for dataset 1
Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.2
Loading data from file and computing dataframes


/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/media/controlslab/DATA/Projects/NASA_RUL_-CMAPS-/code/data_handler_CMAPS.py:133: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  norm_test_df = pd.DataFrame(self._data_scaler.transform(self._df_test[cols_normalize]), columns=cols_normalize, index=self._df_test.index)


cv number
[91, 40, 33, 2, 73, 24, 78, 76, 6, 21, 65, 48, 11, 79, 1, 58, 51, 60, 88, 57]
training with cv
CV set
20/20 [==============================] - 0s 92us/step
Test set
10/10 [==============================] - 0s 97us/step
training with cv
CV set
20/20 [==============================] - 0s 99us/step
Test set
10/10 [==============================] - 0s 101us/step
training with cv
CV set
20/20 [==============================] - 0s 94us/step
Test set
10/10 [==============================] - 0s 102us/step
training with cv
CV set
20/20 [==============================] - 0s 97us/step
Test set
10/10 [==============================] - 0s 103us/step
training with cv
CV set
20/20 [==============================] - 0s 88us/step
Test set
10/10 [==============================] - 0s 106us/step
training with cv
CV set
20/20 [==============================] - 0s 103us/step
Test set
10/10 [==============================] - 0s 109us/step
training with cv
CV set
20/20 [=============================

# Test on all Datasets

In [9]:
# datasets = [1,2,3,4]
# iterations = 10
# tModel.epochs = 150
# lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
# scores ={1:[], 2:[], 3:[], 4:[]}
# window_sizes = {1:17,2:17,3:17,4:17}
# strides = {1:1,2:1,3:1,4:1}
# max_ruls = {1:129, 2:129, 3:129, 4:129}
# num_features = len(selected_features)

# input_shape = None
# models = {'shallow-20-20':RULmodel_SN_4}

# #For each model
# for key, model_def in models.items():
#     file = open("results/MLP/ResultsDatasets_singleSet"+key+".csv", "w")
#     csvfile = csv.writer(file, lineterminator='\n')
    
#     print("Generating statistics for model " + key)

#     #For each dataset
#     for i in range(1,5):
        
#         print("Working on dataset " + str(i))
        
#         tempScoresRMSE = np.zeros((iterations,1))
#         tempScoresRHS = np.zeros((iterations,1))
#         tempTime = np.zeros((iterations,1))
        
#         input_shape = window_sizes[i]*num_features #For simple ANN
#         #input_shape = (window_sizes[i],num_features) #For RNN
        
#         print(input_shape)
        
#         tModel.data_handler.change_dataset(i)
#         tModel.data_handler.sequence_length = window_sizes[i]
#         tModel.data_handler.sequence_stride = strides[i]
#         tModel.data_handler.max_rul = max_ruls[i]
#         tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
#         #tModel.print_data()
        
#         #tModel.print_data()
        
#         for j in range(iterations):

#             #Model needs to be recompiled everytime since they are different runs so weights should be reinit
#             model = get_compiled_model(model_def, input_shape, model_type='ann')

#             tModel.change_model(key, model, 'keras')
#             tModel.train_model(learningRate_scheduler=lrate, verbose=0)
#             tModel.evaluate_model(['rhs', 'rmse'], round=2)
#             #print("scores")
            
#             #print(j)

#             cScores = tModel.scores
#             rmse = math.sqrt(cScores['score_1'])
#             rmse2 = cScores['rmse']
#             rhs = cScores['rhs']
#             time = tModel.train_time
            
#             tempScoresRMSE[j] = rmse2
#             tempScoresRHS[j] = rhs
#             tempTime[j] = time
            
#         print("Results for model " + key)
    
#         print(stats.describe(tempScoresRMSE))
#         print(stats.describe(tempScoresRHS))
#         print(stats.describe(tempTime))
            
#         tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
#         tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
#         tempTime = np.reshape(tempTime, (iterations,))
#         csvfile.writerow(tempScoresRMSE)
#         csvfile.writerow(tempScoresRHS)
#         csvfile.writerow(tempTime)
    
#     file.close()